In [7]:

from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential, load_model # type: ignore
from keras.layers import LSTM, Dense, Dropout # type: ignore
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import requests


In [19]:
# Orijinal veri dosyası yolu
csv_path = '/Users/tbai/Documents/Vsanaliz/analiz/project/ABD/Data/sp500_data.csv'
df = pd.read_csv(csv_path, header=[0, 1])

# Farklı hisse sembollerini içeren dosya yolu
symbols_path = "/Users/tbai/Documents/Vsanaliz/analiz/output/farkli_hisseler.csv"
symbols_df = pd.read_csv(symbols_path)

# Farklı hisselerin sembollerini alın
selected_symbols = symbols_df['FarkliHisseler'].tolist()

# Seçilen sembollere ait Close sütunlarını bulun
close_columns = [col for col in df.columns if col[0] in selected_symbols and col[1] == 'Close']

# Seçilen Close sütunlarını ve hisse adlarını kontrol etme
print(f"Seçilen Close sütunları: {close_columns}")

save_dir = "trained_models_ABD"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

lookback = 8

for col in close_columns:
    symbol = col[0]
    print(f"Eğitim yapılıyor: {symbol}")

    close_data = df[col].dropna().values.reshape(-1, 1).astype(float)
    
    train_size = int(len(close_data) * 0.9)
    train_close = close_data[:train_size]
    test_close = close_data[train_size:]

    scaler = MinMaxScaler(feature_range=(0, 1))
    data_training_array = scaler.fit_transform(train_close)
    

    x_train, y_train = [], []
    for i in range(lookback, len(data_training_array)):
        x_train.append(data_training_array[i-lookback:i])
        y_train.append(data_training_array[i, 0])
    x_train, y_train = np.array(x_train), np.array(y_train)
    
    model = Sequential()
    model.add(LSTM(units=50, activation='relu', return_sequences=True, input_shape=(x_train.shape[1], 1)))
    model.add(Dropout(0.2))
    model.add(LSTM(units=60, activation='relu', return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(units=80, activation='relu', return_sequences=True))
    model.add(Dropout(0.4))
    model.add(LSTM(units=120, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(units=1))
    
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=[tf.keras.metrics.MeanAbsoluteError()])
    model.fit(x_train, y_train, epochs=32, verbose=1)
    
  
    model_path = os.path.join(save_dir, f"{symbol}_model.h5")
    model.save(model_path)
    print(f"Model kaydedildi: {model_path}")
    
print("Tüm işlemler başarıyla sonuçlandı")

/var/folders/0v/wrj2cc8j4px7b649g30sdv2w0000gn/T/ipykernel_74730/461679605.py:3: DtypeWarning: Columns (78,379,393,414,673,876,904,995,1177,1268,1394,1478,1485,1492,1905,2199,2549,2815,2934,2948,3284,3319,3396,3466) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path, header=[0, 1])


Seçilen Close sütunları: [('O', 'Close'), ('WAB', 'Close'), ('WBA', 'Close'), ('WMT', 'Close'), ('DIS', 'Close'), ('WBD', 'Close'), ('WM', 'Close'), ('WAT', 'Close'), ('WEC', 'Close'), ('WFC', 'Close'), ('WELL', 'Close'), ('WST', 'Close'), ('WDC', 'Close'), ('WY', 'Close'), ('WMB', 'Close'), ('WTW', 'Close'), ('WYNN', 'Close'), ('XEL', 'Close'), ('XYL', 'Close'), ('YUM', 'Close'), ('ZBRA', 'Close'), ('ZBH', 'Close'), ('ZTS', 'Close')]
Eğitim yapılıyor: O


/Users/tbai/Documents/Vsanaliz/analiz/vproject2/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/32
36/36 ━━━━━━━━━━━━━━━━━━━━ 32s 549ms/step - loss: 0.2008 - mean_absolute_error: 0.3870
Epoch 2/32
36/36 ━━━━━━━━━━━━━━━━━━━━ 12s 334ms/step - loss: 0.0256 - mean_absolute_error: 0.1247
Epoch 3/32
36/36 ━━━━━━━━━━━━━━━━━━━━ 12s 330ms/step - loss: 0.0157 - mean_absolute_error: 0.0957
Epoch 4/32
36/36 ━━━━━━━━━━━━━━━━━━━━ 12s 330ms/step - loss: 0.0192 - mean_absolute_error: 0.1101
Epoch 5/32
36/36 ━━━━━━━━━━━━━━━━━━━━ 12s 324ms/step - loss: 0.0128 - mean_absolute_error: 0.0867
Epoch 6/32
36/36 ━━━━━━━━━━━━━━━━━━━━ 12s 321ms/step - loss: 0.0127 - mean_absolute_error: 0.0873
Epoch 7/32
36/36 ━━━━━━━━━━━━━━━━━━━━ 12s 319ms/step - loss: 0.0104 - mean_absolute_error: 0.0765
Epoch 8/32
36/36 ━━━━━━━━━━━━━━━━━━━━ 12s 320ms/step - loss: 0.0110 - mean_absolute_error: 0.0807
Epoch 9/32
36/36 ━━━━━━━━━━━━━━━━━━━━ 11s 317ms/step - loss: 0.0099 - mean_absolute_error: 0.0736
Epoch 10/32
36/36 ━━━━━━━━━━━━━━━━━━━━ 12s 320ms/step - loss: 0.0112 - mean_absolute_error: 0.0790
Epoch 11/32
36/36 ━

Model kaydedildi: trained_models_ABD/O_model.h5
Eğitim yapılıyor: WAB


...

36/36 ━━━━━━━━━━━━━━━━━━━━ 33s 585ms/step - loss: 0.2224 - mean_absolute_error: 0.4147
Epoch 2/32
36/36 ━━━━━━━━━━━━━━━━━━━━ 12s 337ms/step - loss: 0.0272 - mean_absolute_error: 0.1324
Epoch 3/32
36/36 ━━━━━━━━━━━━━━━━━━━━ 12s 335ms/step - loss: 0.0157 - mean_absolute_error: 0.0976
Epoch 4/32
36/36 ━━━━━━━━━━━━━━━━━━━━ 12s 335ms/step - loss: 0.0141 - mean_absolute_error: 0.0900
Epoch 5/32
36/36 ━━━━━━━━━━━━━━━━━━━━ 12s 330ms/step - loss: 0.0123 - mean_absolute_error: 0.0873
Epoch 6/32
36/36 ━━━━━━━━━━━━━━━━━━━━ 12s 331ms/step - loss: 0.0121 - mean_absolute_error: 0.0841
Epoch 7/32
36/36 ━━━━━━━━━━━━━━━━━━━━ 12s 327ms/step - loss: 0.0110 - mean_absolute_error: 0.0796
Epoch 8/32
36/36 ━━━━━━━━━━━━━━━━━━━━ 12s 328ms/step - loss: 0.0092 - mean_absolute_error: 0.0734
Epoch 9/32
36/36 ━━━━━━━━━━━━━━━━━━━━ 12s 322ms/step - loss: 0.0103 - mean_absolute_error: 0.0779
Epoch 10/32
36/36 ━━━━━━━━━━━━━━━━━━━━ 12s 326ms/step - loss: 0.0114 - mean_absolute_error: 0.0824
Epoch 11/32
36/36 ━━━━━━━━━━━━

Model kaydedildi: trained_models_ABD/ZBH_model.h5
Eğitim yapılıyor: ZTS


/Users/tbai/Documents/Vsanaliz/analiz/vproject2/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/32
36/36 ━━━━━━━━━━━━━━━━━━━━ 35s 617ms/step - loss: 0.1560 - mean_absolute_error: 0.3391
Epoch 2/32
36/36 ━━━━━━━━━━━━━━━━━━━━ 12s 341ms/step - loss: 0.0196 - mean_absolute_error: 0.1118
Epoch 3/32
36/36 ━━━━━━━━━━━━━━━━━━━━ 12s 335ms/step - loss: 0.0105 - mean_absolute_error: 0.0779
Epoch 4/32
36/36 ━━━━━━━━━━━━━━━━━━━━ 12s 332ms/step - loss: 0.0069 - mean_absolute_error: 0.0636
Epoch 5/32
36/36 ━━━━━━━━━━━━━━━━━━━━ 12s 327ms/step - loss: 0.0071 - mean_absolute_error: 0.0659
Epoch 6/32
36/36 ━━━━━━━━━━━━━━━━━━━━ 12s 336ms/step - loss: 0.0070 - mean_absolute_error: 0.0640
Epoch 7/32
36/36 ━━━━━━━━━━━━━━━━━━━━ 12s 324ms/step - loss: 0.0062 - mean_absolute_error: 0.0597
Epoch 8/32
36/36 ━━━━━━━━━━━━━━━━━━━━ 12s 331ms/step - loss: 0.0068 - mean_absolute_error: 0.0637
Epoch 9/32
36/36 ━━━━━━━━━━━━━━━━━━━━ 12s 330ms/step - loss: 0.0068 - mean_absolute_error: 0.0629
Epoch 10/32
36/36 ━━━━━━━━━━━━━━━━━━━━ 12s 329ms/step - loss: 0.0057 - mean_absolute_error: 0.0577
Epoch 11/32
36/36 ━

Model kaydedildi: trained_models_ABD/ZTS_model.h5
Tüm işlemler başarıyla sonuçlandı


In [8]:
csv_path = '/Users/tbai/Documents/Vsanaliz/analiz/project/ABD/Data/sp500_data.csv'
df = pd.read_csv(csv_path)

/var/folders/0v/wrj2cc8j4px7b649g30sdv2w0000gn/T/ipykernel_13074/3807279675.py:2: DtypeWarning: Columns (2,3,4,5,6,7,9,10,11,12,13,14,16,17,18,19,20,21,23,24,25,26,27,28,30,31,32,33,34,35,37,38,39,40,41,42,44,45,46,47,48,49,51,52,53,54,55,56,58,59,60,61,62,63,65,66,67,68,69,70,72,73,74,75,76,77,78,79,80,81,82,83,84,86,87,88,89,90,91,93,94,95,96,97,98,100,101,102,103,104,105,107,108,109,110,111,112,114,115,116,117,118,119,121,122,123,124,125,126,128,129,130,131,132,133,135,136,137,138,139,140,142,143,144,145,146,147,149,150,151,152,153,154,156,157,158,159,160,161,163,164,165,166,167,168,170,171,172,173,174,175,177,178,179,180,181,182,184,185,186,187,188,189,191,192,193,194,195,196,198,199,200,201,202,203,205,206,207,208,209,210,212,213,214,215,216,217,219,220,221,222,223,224,226,227,228,229,230,231,233,234,235,236,237,238,240,241,242,243,244,245,247,248,249,250,251,252,254,255,256,257,258,259,261,262,263,264,265,266,268,269,270,271,272,273,275,276,277,278,279,280,282,283,284,285,286,287

In [9]:
df

,Unnamed: 0,MMM,MMM.1,MMM.2,MMM.3,MMM.4,MMM.5,MMM.6,AOS,AOS.1,...,ZBH.4,ZBH.5,ZBH.6,ZTS,ZTS.1,ZTS.2,ZTS.3,ZTS.4,ZTS.5,ZTS.6
0,NaN,Date,Open,High,Low,Close,Volume,VWAP,Date,Open,...,Close,Volume,VWAP,Date,Open,High,Low,Close,Volume,VWAP
1,0.0,2019-08-22 04:00:00,135.4515,136.2291,133.9047,134.5151,1750237.164,134.7568,2019-08-22 04:00:00,46.99,...,137.11,678128.0,137.164,2019-08-22 04:00:00,127.88,128.11,125.52,126.64,2426745.0,126.5754
2,1.0,2019-08-23 04:00:00,133.8462,134.0719,129.8244,130.3094,5085268.812,130.9103,2019-08-23 04:00:00,46.26,...,133.64,1284110.0,134.5661,2019-08-23 04:00:00,126.58,127.49,122.42,123.07,2355068.0,124.1573
3,2.0,2019-08-26 04:00:00,131.2793,131.6597,129.6656,130.5351,2696519.54,130.4074,2019-08-26 04:00:00,45.42,...,134.83,897804.0,134.5019,2019-08-26 04:00:00,124.31,126.24,124.2468,126.2,1285981.0,125.5652
4,3.0,2019-08-27 04:00:00,131.5468,131.6722,129.9331,130.2258,2774161.468,130.4286,2019-08-27 04:00:00,45.35,...,134.5,803156.0,134.8981,2019-08-27 04:00:00,127.01,127.47,124.75,125.24,2424435.0,125.736
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1257,1256.0,2024-08-20 04:00:00,126.99,128.0653,126.88,127.7,2756006.0,127.542,2024-08-20 04:00:00,80.63,...,111.64,870512.0,111.7635,2024-08-20 04:00:00,184.48,184.76,182.9001,183.6,1108171.0,183.592
1258,1257.0,2024-08-21 04:00:00,127.84,129.99,127.66,129.23,3472238.0,129.2094,2024-08-21 04:00:00,81.19,...,112.13,796201.0,111.8592,2024-08-21 04:00:00,184.27,184.525,181.82,182.9,1212039.0,182.9182
1259,1258.0,2024-08-22 04:00:00,129.1,129.75,128.3,128.6,2261878.0,128.8922,2024-08-22 04:00:00,81.82,...,113.42,1141213.0,113.1862,2024-08-22 04:00:00,183.41,184.3475,181.51,182.17,1121690.0,182.4637
1260,1259.0,2024-08-23 04:00:00,129.23,130.65,128.85,130.55,2522332.0,129.7872,2024-08-23 04:00:00,81.82,...,115.05,772614.0,114.8907,2024-08-23 04:00:00,182.86,182.9381,180.18,180.9,1162538.0,181.1325


In [10]:
df = pd.read_csv('/Users/tbai/Documents/Vsanaliz/analiz/project/ABD/Data/sp500_data.csv', header=[0, 1])
close_columns = [col for col in df.columns if col[1] == 'Close']

# Seçilen Close sütunlarını ve hisse adlarını kontrol etme
print(f"Seçilen Close sütunları: {close_columns}")

Seçilen Close sütunları: [('MMM', 'Close'), ('AOS', 'Close'), ('ABT', 'Close'), ('ABBV', 'Close'), ('ACN', 'Close'), ('ADBE', 'Close'), ('AMD', 'Close'), ('AES', 'Close'), ('AFL', 'Close'), ('A', 'Close'), ('APD', 'Close'), ('ABNB', 'Close'), ('AKAM', 'Close'), ('ALB', 'Close'), ('ARE', 'Close'), ('ALGN', 'Close'), ('ALLE', 'Close'), ('LNT', 'Close'), ('ALL', 'Close'), ('GOOGL', 'Close'), ('GOOG', 'Close'), ('MO', 'Close'), ('AMZN', 'Close'), ('AMCR', 'Close'), ('AEE', 'Close'), ('AAL', 'Close'), ('AEP', 'Close'), ('AXP', 'Close'), ('AIG', 'Close'), ('AMT', 'Close'), ('AWK', 'Close'), ('AMP', 'Close'), ('AME', 'Close'), ('AMGN', 'Close'), ('APH', 'Close'), ('ADI', 'Close'), ('ANSS', 'Close'), ('AON', 'Close'), ('APA', 'Close'), ('AAPL', 'Close'), ('AMAT', 'Close'), ('APTV', 'Close'), ('ACGL', 'Close'), ('ADM', 'Close'), ('ANET', 'Close'), ('AJG', 'Close'), ('AIZ', 'Close'), ('T', 'Close'), ('ATO', 'Close'), ('ADSK', 'Close'), ('ADP', 'Close'), ('AZO', 'Close'), ('AVB', 'Close'), ('AVY'

/var/folders/0v/wrj2cc8j4px7b649g30sdv2w0000gn/T/ipykernel_13074/1501400489.py:1: DtypeWarning: Columns (78,379,393,414,673,876,904,995,1177,1268,1394,1478,1485,1492,1905,2199,2549,2815,2934,2948,3284,3319,3396,3466) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/Users/tbai/Documents/Vsanaliz/analiz/project/ABD/Data/sp500_data.csv', header=[0, 1])


In [ ]:
save_dir = "trained_models_ABD"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

lookback = 8

for col in close_columns:
    symbol = col[0]  
    print(f"Eğitim yapılıyor: {symbol}")

    close_data = df[col].dropna().values.reshape(-1, 1).astype(float)
    
    train_size = int(len(close_data) * 0.9)
    train_close = close_data[:train_size]
    test_close = close_data[train_size:]

    scaler = MinMaxScaler(feature_range=(0, 1))
    data_training_array = scaler.fit_transform(train_close)
    

    x_train, y_train = [], []
    for i in range(lookback, len(data_training_array)):
        x_train.append(data_training_array[i-lookback:i])
        y_train.append(data_training_array[i, 0])
    x_train, y_train = np.array(x_train), np.array(y_train)
    
    model = Sequential()
    model.add(LSTM(units=50, activation='relu', return_sequences=True, input_shape=(x_train.shape[1], 1)))
    model.add(Dropout(0.2))
    model.add(LSTM(units=60, activation='relu', return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(units=80, activation='relu', return_sequences=True))
    model.add(Dropout(0.4))
    model.add(LSTM(units=120, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(units=1))
    
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=[tf.keras.metrics.MeanAbsoluteError()])
    model.fit(x_train, y_train, epochs=32, verbose=1)
    
  
    model_path = os.path.join(save_dir, f"{symbol}_model.h5")
    model.save(model_path)
    print(f"Model kaydedildi: {model_path}")
    
print("tüm işlemler başarıyla sonuçlandı")

Eğitim yapılıyor: MMM
Epoch 1/32


/Users/tbai/Documents/Vsanaliz/analiz/vproject2/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


36/36 ━━━━━━━━━━━━━━━━━━━━ 16s 343ms/step - loss: 0.2159 - mean_absolute_error: 0.3800
Epoch 2/32
36/36 ━━━━━━━━━━━━━━━━━━━━ 11s 310ms/step - loss: 0.0293 - mean_absolute_error: 0.1398
Epoch 3/32
36/36 ━━━━━━━━━━━━━━━━━━━━ 11s 304ms/step - loss: 0.0119 - mean_absolute_error: 0.0860
Epoch 4/32
36/36 ━━━━━━━━━━━━━━━━━━━━ 10s 286ms/step - loss: 0.0101 - mean_absolute_error: 0.0769
Epoch 5/32
36/36 ━━━━━━━━━━━━━━━━━━━━ 10s 284ms/step - loss: 0.0104 - mean_absolute_error: 0.0770
Epoch 6/32
36/36 ━━━━━━━━━━━━━━━━━━━━ 10s 283ms/step - loss: 0.0103 - mean_absolute_error: 0.0772
Epoch 7/32
36/36 ━━━━━━━━━━━━━━━━━━━━ 10s 285ms/step - loss: 0.0099 - mean_absolute_error: 0.0736
Epoch 8/32
36/36 ━━━━━━━━━━━━━━━━━━━━ 10s 283ms/step - loss: 0.0096 - mean_absolute_error: 0.0735
Epoch 9/32
36/36 ━━━━━━━━━━━━━━━━━━━━ 10s 291ms/step - loss: 0.0087 - mean_absolute_error: 0.0711
Epoch 10/32
36/36 ━━━━━━━━━━━━━━━━━━━━ 10s 288ms/step - loss: 0.0089 - mean_absolute_error: 0.0721
Epoch 11/32
36/36 ━━━━━━━━━━━━

Model kaydedildi: trained_models_ABD/MMM_model.h5
Eğitim yapılıyor: AOS
Epoch 1/32


/Users/tbai/Documents/Vsanaliz/analiz/vproject2/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

/Users/tbai/Documents/Vsanaliz/analiz/vproject2/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/32
36/36 ━━━━━━━━━━━━━━━━━━━━ 63s 1s/step - loss: 0.1204 - mean_absolute_error: 0.2759
Epoch 2/32
36/36 ━━━━━━━━━━━━━━━━━━━━ 13s 365ms/step - loss: 0.0154 - mean_absolute_error: 0.0967
Epoch 3/32
36/36 ━━━━━━━━━━━━━━━━━━━━ 13s 347ms/step - loss: 0.0143 - mean_absolute_error: 0.0867
Epoch 4/32
36/36 ━━━━━━━━━━━━━━━━━━━━ 13s 348ms/step - loss: 0.0084 - mean_absolute_error: 0.0661
Epoch 5/32
36/36 ━━━━━━━━━━━━━━━━━━━━ 13s 354ms/step - loss: 0.0087 - mean_absolute_error: 0.0648
Epoch 6/32
36/36 ━━━━━━━━━━━━━━━━━━━━ 12s 346ms/step - loss: 0.0078 - mean_absolute_error: 0.0614
Epoch 7/32
36/36 ━━━━━━━━━━━━━━━━━━━━ 12s 338ms/step - loss: 0.0075 - mean_absolute_error: 0.0602
Epoch 8/32
36/36 ━━━━━━━━━━━━━━━━━━━━ 12s 333ms/step - loss: 0.0076 - mean_absolute_error: 0.0606
Epoch 9/32
36/36 ━━━━━━━━━━━━━━━━━━━━ 12s 345ms/step - loss: 0.0055 - mean_absolute_error: 0.0528
Epoch 10/32
36/36 ━━━━━━━━━━━━━━━━━━━━ 12s 335ms/step - loss: 0.0061 - mean_absolute_error: 0.0559
Epoch 11/32
36/36 ━━━━

Model kaydedildi: trained_models_ABD/FSLR_model.h5
Eğitim yapılıyor: FE


/Users/tbai/Documents/Vsanaliz/analiz/vproject2/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/32
36/36 ━━━━━━━━━━━━━━━━━━━━ 63s 1s/step - loss: 0.1692 - mean_absolute_error: 0.3514
Epoch 2/32
36/36 ━━━━━━━━━━━━━━━━━━━━ 13s 363ms/step - loss: 0.0242 - mean_absolute_error: 0.1208
Epoch 3/32
36/36 ━━━━━━━━━━━━━━━━━━━━ 13s 352ms/step - loss: 0.0151 - mean_absolute_error: 0.0946
Epoch 4/32
36/36 ━━━━━━━━━━━━━━━━━━━━ 13s 353ms/step - loss: 0.0133 - mean_absolute_error: 0.0865
Epoch 5/32
36/36 ━━━━━━━━━━━━━━━━━━━━ 12s 341ms/step - loss: 0.0106 - mean_absolute_error: 0.0764
Epoch 6/32
36/36 ━━━━━━━━━━━━━━━━━━━━ 13s 347ms/step - loss: 0.0096 - mean_absolute_error: 0.0754
Epoch 7/32
36/36 ━━━━━━━━━━━━━━━━━━━━ 12s 339ms/step - loss: 0.0097 - mean_absolute_error: 0.0723
Epoch 8/32
36/36 ━━━━━━━━━━━━━━━━━━━━ 12s 333ms/step - loss: 0.0103 - mean_absolute_error: 0.0783
Epoch 9/32
36/36 ━━━━━━━━━━━━━━━━━━━━ 12s 325ms/step - loss: 0.0104 - mean_absolute_error: 0.0756
Epoch 10/32
36/36 ━━━━━━━━━━━━━━━━━━━━ 12s 331ms/step - loss: 0.0119 - mean_absolute_error: 0.0818
Epoch 11/32
36/36 ━━━━

In [11]:
print(f"{len(close_columns)} adet hisse mevcut.")

503 adet hisse mevcut.


In [12]:
save_dir = "trained_models_ABD"
h5_files = [f for f in os.listdir(save_dir) if f.endswith('.h5')]
h5_file_count = len(h5_files)

print(f"{save_dir} klasöründe {h5_file_count} adet .h5 dosyası var.")

# Sembolleri listele
symbols = [os.path.splitext(f)[0].replace('_model', '') for f in h5_files]
print("Kayıt edilen semboller:")
for symbol in symbols:
    print(symbol)

trained_models_ABD klasöründe 503 adet .h5 dosyası var.
Kayıt edilen semboller:
BG
NUE
GPC
VST
FDX
LUV
SYF
FIS
CTVA
TGT
EXPE
EXPD
FTNT
GNRC
SWKS
HAS
WMB
DHR
MOH
EA
CPT
CME
ATO
VTR
HST
SO
UDR
BSX
GOOGL
HPQ
WYNN
CZR
HRL
ROK
FE
AWK
NDAQ
ETR
KO
HII
FDS
IR
MLM
STLD
TSCO
NWS
IEX
LH
CPRT
REG
OTIS
LVS
PWR
TRMB
IDXX
XOM
COR
AVY
BF.B
BKR
BIO
FICO
WM
AMAT
ROST
ABT
L
WBA
LW
BMY
HOLX
URI
AMZN
JCI
NCLH
MRNA
RCL
TTWO
GEV
JBHT
CARR
TRGP
EPAM
BLK
PAYX
GILD
PTC
KEYS
DG
MCD
IPG
FTV
KIM
STX
PFE
AES
YUM
CSGP
CAH
BR
MGM
DTE
VICI
ORLY
ARE
NKE
HAL
SJM
WY
WFC
QCOM
TFX
COF
MSCI
F
DXCM
BX
FOX
FAST
POOL
LHX
MCO
ABBV
MAS
SYY
MAR
OXY
META
HPE
MO
ES
GDDY
EXR
UPS
GWW
ULTA
J
BLDR
K
GRMN
CVS
CF
PNR
AMD
APTV
AME
LULU
CTSH
MRK
ABNB
BEN
NSC
PARA
SBAC
CRWD
CEG
TSLA
MTCH
VRSK
NEM
KLAC
EMR
DIS
KMB
PPL
CPAY
NDSN
ODFL
PEG
SHW
INTU
KEY
LYV
ANET
INVH
MMM
ADM
NVR
CVX
AOS
PLD
CL
SW
TMUS
EQIX
A
GE
GD
ALLE
PHM
AIG
LKQ
IBM
MSFT
STT
GEHC
UHS
FMC
PPG
KMI
TECH
CBRE
HON
WAB
UBER
AAPL
ENPH
RMD
SOLV
TJX
PYPL
AXP
BA
LNT
ALB
PG
XEL
FFIV
EFX


In [13]:
sembol = pd.DataFrame(symbols, columns=['HisseSembolu'])
csv_path = "kayitli_hisseler.csv"
sembol.to_csv(csv_path, index=False)

print(f"Semboller {csv_path} dosyasına kaydedildi.")

Semboller kayitli_hisseler.csv dosyasına kaydedildi.
